In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
def hex_to_rgba(h, alpha):
  return "rgba" + str(tuple([int(h.lstrip('#')[i:i+2], 16) for i in (0, 2, 4)] + [alpha]))

def parse_results_file(fname, labels):
  with open(fname, "r") as f:
    data = f.read()
  df_data = {
    "time": []
  }
  for label in labels:
    df_data[label] = []

  header_is_next = False
  header_read = False
  for line in data.split("\n"):
    if not line:
      continue
    if header_is_next:
      header = line
      header_read = True
      header_is_next = False
    elif "*" in line:
      header_is_next = not header_read
    else:
      parts = [part.strip() for part in line.split("|")]
      for part in parts:
        x_label = part.split("=")[0]
        if x_label in labels:
          x_val = float(part.split("=")[-1])
          df_data[x_label].append(x_val)
      y_val = float(parts[-1])
      df_data['time'].append(y_val)
  return header, pd.DataFrame(data=df_data)

## MPI scaling

In [3]:
header, df = parse_results_file("mpiscaling/20240510_105932_2epyc_mpiscaling.txt", ["nodes"])

In [4]:
fig = make_subplots()
fig.update_layout(
  legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
  ),
  xaxis_title="np (CPU cores)",
  yaxis_title="Time"
)
fig.add_trace(go.Scatter(
  x=df["nodes"], 
  y=df["time"],
  mode='lines+markers', 
  name=f"MPI scaling, --map-by socket"
))
fig.add_trace(go.Scatter(
  x=df["nodes"], 
  y=df["nodes"].apply(lambda np: df["time"].iloc[-1] / np * df["nodes"].iloc[-1]),
  mode='lines+markers', 
  line=dict(
    dash='dot'
  ),
  name="Theoretical scalability"
))

fig.update_xaxes(type='log')
fig.update_yaxes(type='log')
fig

## Opt scaling

In [5]:
header, df = parse_results_file("opt/20240509_235051_2epyc_opt.txt", ["places", "nodes"])

In [7]:
symbols = ["circle", "square", "x", "star"]
colors = ["#EBD9B4", "#75A47F", "#E6A4B4", "#756AB6"]

fig = make_subplots()
fig.update_layout(
  legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
  ),
  xaxis_title="np (CPU cores)",
  yaxis_title="Time"
)
by_node = df.groupby("nodes").aggregate(lambda x: x.to_list())
for x in range(4):
  fig.add_trace(go.Scatter(
    x=by_node.iloc[x]["places"], 
    y=by_node.iloc[x]["time"], 
    mode='lines+markers', 
    name=f"{x+1} {'socket' if x+1 == 1 else 'sockets'}",
    marker=dict(
      symbol=symbols[x],color=colors[x],
    )
  ))
  fig.add_trace(go.Scatter(
    x=by_node.iloc[x]["places"], 
    y=[by_node.iloc[x]["time"][-1] / np * by_node.iloc[x]["places"][-1] for np in by_node.iloc[x]["places"]],
    mode='lines+markers', 
    name=f"Theoretical speedup with {x+1} {'socket' if x+1 == 1 else 'sockets'}",
    marker=dict(
      symbol=symbols[x],
      color=colors[x]
    ),
    line=dict(
      color=hex_to_rgba(colors[x], 0.5),
      dash='dot'
    )
  ))
fig.update_xaxes(type='log')
fig.update_yaxes(type='log')
fig

## OMP scaling

In [8]:
header, df = parse_results_file("ompscaling/20240515_152522_2epyc_ompscaling.txt", ["places"])

In [9]:
header, df1 = parse_results_file("ompscaling/20240510_093852_2epyc_ompscaling.txt", ["places"])

In [10]:
fig = make_subplots()
fig.update_layout(
  legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
  ),
  xaxis_title="Places (CPU cores)",
  yaxis_title="Time"
)
fig.add_trace(go.Scatter(
  x=df["places"], 
  y=df["time"],
  mode='lines+markers', 
  name=f"OMP scaling, OMP_PROC_BIND=spread"
))
fig.add_trace(go.Scatter(
  x=df1["places"], 
  y=df1["time"],
  mode='lines+markers', 
  name=f"OMP scaling, OMP_PROC_BIND=close"
))
fig.add_trace(go.Scatter(
  x=df["places"], 
  y=df["places"].apply(lambda np: df["time"].iloc[-1] / np * df["places"].iloc[-1]),
  mode='lines+markers', 
  line=dict(
    dash='dot'
  ),
  name="Theoretical speedup"
))

fig.update_xaxes(type='log')
fig.update_yaxes(type='log')
fig

## OMP scaling (small)

In [11]:
header, df1 = parse_results_file("ompscaling/20240519_104329_2epyc_ompscaling.txt", ["places"])

In [12]:
header, df = parse_results_file("ompscaling/20240519_104732_2epyc_ompscaling.txt", ["places"])

In [17]:
fig = make_subplots()
fig.update_layout(
  legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
  ),
  xaxis_title="Places (CPU cores)",
  yaxis_title="Time"
)
fig.add_trace(go.Scatter(
  x=df["places"], 
  y=df["time"],
  mode='lines+markers', 
  name=f"OMP scaling, OMP_PROC_BIND=spread"
))
fig.add_trace(go.Scatter(
  x=df1["places"], 
  y=df1["time"],
  mode='lines+markers', 
  name=f"OMP scaling, OMP_PROC_BIND=close"
))
fig.add_trace(go.Scatter(
  x=df["places"], 
  y=df["places"].apply(lambda np: df["time"].iloc[-1] / np * df["places"].iloc[-1]),
  mode='lines+markers', 
  line=dict(
    dash='dot'
  ),
  name="Theoretical speedup OMP_PROC_BIND=spread"
))
fig.add_trace(go.Scatter(
  x=df["places"], 
  y=df1["places"].apply(lambda np: df1["time"].iloc[-1] / np * df1["places"].iloc[-1]),
  mode='lines+markers', 
  line=dict(
    dash='dot'
  ),
  name="Theoretical speedup OMP_PROC_BIND=close"
))

fig.update_xaxes(type='log')
fig.update_yaxes(type='log')
fig


## Weak scaling

In [14]:
header, df = parse_results_file("weak/20240522_194423_epyc_weak.txt", ["places", "ny"])

In [16]:
fig = make_subplots()
fig.update_layout(
  legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
  ),
  xaxis_title="x size, places (cpu cores) halved at every step starting from 128",
  yaxis_title="Time"
)
fig.add_trace(go.Scatter(
  x=df["ny"], 
  y=df["time"],
  mode='lines+markers', 
  name=f"Weak scaling"
))

fig.update_xaxes(type='linear')
fig.update_yaxes(type='linear')
fig